# Poetry Notebook

In this notebook we will be implementing GPT to generate text based on the work of Edgar Allan Poe.

In [1]:
# Installing dependencies
!pip install tiktoken
!pip install torch

# Downloading dataset from the GitHub
!wget https://raw.githubusercontent.com/kocenko/Poetry-Synthesis/main/data/poe_data.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-17 15:08:51--  https://raw.githubusercontent.com/kocenko/Poetry-Synthesis/dev/data/poe_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1930488 (1.8M) [text/plain]
Saving to: ‘poe_data.txt’

poe_data.txt        100%[===================>]   1.84M  --.-KB/s    in 0.007s  

2023-05-17 15:08:53 (280 MB/s) - ‘poe_data.txt’ saved [1930488/1930488]



In [2]:
# Essential imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [3]:
# Testing if GPU is available
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

CUDA_LAUNCH_BLOCKING=1

In [4]:
# Dataset class definition
### (Option) We can use different data to train it on
### (Option) What if the context affects not the following
###          but the one after the following token? (bigger offset)

class PoeDataset(Dataset):
    valid_split_params = ["train", "valid"]

    def __init__(self, text: str, split: str, split_ratio: float, context_length: int, tokenizer, offset: int = 1):
        ''' Poe Dataset constructor

        Args:
            str:
                file_path: Path to the file containing dataset
                splt: String indicating what type of data this dataset contains
            float:
                split_ratio: Value between (0, 1] of what should be the ratio
                             between training and validation set
            int:
                context_length: Length of the context
                offset: An offset between the end of the context and the target
        '''

        assert split in PoeDataset.valid_split_params, f"{split} is the wrong split type"
        assert split_ratio <= 1 and split_ratio > 0, f"Split ratio value should be from range (0, 1]"
        assert len(text) > 0, f"Dataset file should not be empty"
        assert context_length < len(text), f"Context length should not be more than {len(text) - 1}"

        self.text = text
        self.offset = offset
        self.context_length = context_length
        self.tokenizer = tokenizer
        self.data = torch.tensor(self.tokenizer.encode(self.text), dtype=torch.int32, device=device)

        split_idx = int(len(self.data) * split_ratio)
        if split == "train":
            self.data = self.data[:split_idx]
        else:
            self.data = self.data[split_idx:]

    def __len__(self):
        ''' Returns the size of the dataset
        
        Returns:
            Number of possible shifts in the dataset for choosing the context chunk
        '''
        return len(self.data) - self.context_length - self.context_length + 1
    
    def __getitem__(self, index):
        ''' Returns an item of given index

        Params:
            index: Which item should be returned
        
        Returns:
            Sample of given index
        '''
        
        x = self.data[index: index + self.context_length]
        y = self.data[index + self.offset: index + self.context_length + self.offset]

        return x, y



In [5]:
# Defined tokenizer class
import torch
from typing import List


class Tokenizer:
    ''' Class for character-wise tokenization'''

    def __init__(self, text: str):
        assert len(text) > 0, "Text used for creating tokenizer cannot be empty"

        self.text = text
        self.symbols = sorted(list(set(self.text)))
        self.vocab_size = len(self.symbols)
        self.stoi = { ch:i for i, ch in enumerate(self.symbols)}
        self.itos = { i:ch for i, ch in enumerate(self.symbols)}

    def encode(self, text: str) -> List:
        ''' Encodes string to list of ints '''

        return [self.stoi[ch] for ch in text]
    
    def decode(self, tokens: List) -> str:
        ''' Decodes list of ints to string '''
        
        return ''.join([self.itos[token] for token in tokens])


In [6]:
from torch.nn.modules.dropout import Dropout
# Simple Decoder Class definition
### (Option) Different split, test data?
from typing import Tuple
import torch.nn.functional as F

class SingleAttentionHead(nn.Module):

    def __init__(self, head_size, config):
        super().__init__()
        self.n_embed = config["n_embed"]
        self.context_length = config["context_length"]
        self.dropout = config["dropout"]

        self.query = nn.Linear(self.n_embed, head_size, bias = False, device=device)
        self.key = nn.Linear(self.n_embed, head_size, bias = False, device=device)
        self.value = nn.Linear(self.n_embed, head_size, bias = False, device=device)
        self.triangle_matrix = torch.tril(torch.ones(self.context_length, self.context_length, device=device))
        self.head_dropout = nn.Dropout(self.dropout)

    def forward(self, x):
        B, T, C = x.shape
        keys = self.key(x)
        query = self.query(x)

        affinities = query @ keys.transpose(-2, -1)  # Dot product, with transposition of T and C
        affinities *= C**(-.5)  # Normalization, to prevent softmax for skewing
        affinities = affinities.masked_fill(self.triangle_matrix[:T, :T] == 0, float('-inf'))
        affinities = F.softmax(affinities, dim=-1)
        affinities = self.head_dropout(affinities)

        v = self.value(x)
        return affinities @ v

class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.heads_num = config["att_head_num"]
        self.n_embed = config["n_embed"]
        self.dropout = config["dropout"]

        head_size = self.n_embed // self.heads_num
        self.attention_heads = nn.ModuleList([SingleAttentionHead(head_size, config) for _ in range(self.heads_num)])
        self.heads_projection = nn.Linear(self.n_embed, self.n_embed, device=device)
        self.heads_dropout = nn.Dropout(self.dropout)
        self.feed_forward = nn.Sequential(nn.Linear(self.n_embed, 4 * self.n_embed, device=device),
                                          nn.ReLU(),
                                          nn.Linear(4 * self.n_embed, self.n_embed, device=device),
                                          nn.Dropout(self.dropout))
        # '4 times n_embed' comes from the paper 'Attention is all you need' (as the whole transformer)
        self.layer_normalization = nn.LayerNorm(self.n_embed, device=device)
        self.layer_normalization2 = nn.LayerNorm(self.n_embed, device=device)

    def forward(self, x):
        x = self.layer_normalization(x)
        x = x + self.heads_dropout(self.heads_projection(torch.cat([att(x) for att in self.attention_heads], dim=-1)))
        x = self.layer_normalization2(x)
        x = x + self.feed_forward(x)
        return x


class OnlyDecoder(nn.Module):
    def __init__(self, config: dict):
        super().__init__()

        self.vocab_size = config["vocab_size"]
        self.n_embed = config["n_embed"]
        self.context_length = config["context_length"]
        self.head_num = config["att_head_num"]
        self.blocks_num = config["blocks_num"]

        self.token_embedding_table = nn.Embedding(self.vocab_size, self.n_embed, device=device)
        self.pos_embedding_table = nn.Embedding(self.context_length, self.n_embed, device=device)
        self.lin = nn.Linear(self.n_embed, self.vocab_size, device=device)
        self.transformer_blocks = nn.Sequential(*[TransformerBlock(config) for _ in range(self.blocks_num)],
                                                nn.LayerNorm(self.n_embed, device=device))
        self.layer_normalization = nn.LayerNorm(self.n_embed, device=device)

    def forward(self, token_idx: int, targets=None):
        B, T = token_idx.shape
        token_embedding = self.token_embedding_table(token_idx)
        pos_embedding = self.pos_embedding_table(torch.arange(T, device=device))
        x = token_embedding + pos_embedding
        x = self.transformer_blocks(x)
        x = self.layer_normalization(x)
        logits = self.lin(x)

        if targets is None:
          loss = None
        else:
          B, T, C = logits.shape
          logits = logits.view(B*T, C)
          targets = targets.view(B*T)
          targets = targets.type(torch.LongTensor).to(device)
          loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate_new_text(self, idx, sym_limit: int) -> torch.Tensor:
        output = []
        for _ in range(sym_limit):
          idx = idx if idx.size(1) <= self.context_length else idx[:, -self.context_length:]
          logits, loss = self(idx)
          logits = logits[:, -1, :]
          probabilities = F.softmax(logits, dim=-1)
          idx_next = torch.multinomial(probabilities, num_samples=1) # Take best
          output.append(idx_next)
          idx = torch.cat((idx, idx_next), dim=1)
        return output


In [7]:
import torch


@torch.no_grad()
def calc_loss(model, iterations, batch_size, train_set, val_set):
    ''' Used to evalute model by averaging on many iterations
    
    Args:
        model: Evaluated model
        iterations: Number of iterations to average through
        batch_size: Batch size
        train_set: Training dataset
        val_set: Validation dataset

    Returns:
        Dictionary with averaged losses for 'train' nad 'valid'
    '''

    split_type = ["train", "valid"]
    outcome_losses = {}
    model.eval()
    for t, split in enumerate([train_set, val_set]):
        loader = DataLoader(split, batch_size = batch_size, shuffle=True, drop_last=True)
        loader = iter(loader)
        losses = torch.zeros(iterations)
        for i in range(iterations):
            x, y = loader.__next__()
            _, loss = model(x, y)
            losses[i] = loss.item()
        outcome_losses[split_type[t]] = losses.mean()
    model.train()
    return outcome_losses


def train_model(model, train_set, valid_set, hyper_params: dict, device):
    ''' Trains the model

    Args:
        model: Model to train
        train_set: Training dataset
        valid_set: Validation dataset
        hyper_params: dict of hyperparameters
    '''

    lr = hyper_params["lr"]
    epochs = hyper_params["epochs"]
    batch_size = hyper_params["batch_size"]
    eval_per_epoch = hyper_params["eval_per_epoch"]
    eval_iterations = hyper_params["eval_iterations"]
    break_iter = hyper_params["break_iter"]
    if break_iter == None:
        break_iter = len(train_set) // batch_size

    train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    eval_each = break_iter // eval_per_epoch
        

    for e in range(epochs):
        for i, (x, y) in enumerate(train_dataloader):
            if i == break_iter:
                break

            if i % eval_each == 0:
                losses = calc_loss(model, eval_iterations, batch_size, train_set, valid_set)
                print(f"Epoch: [{e+1}/{epochs}] Step: [{i}/{break_iter}], train loss: {losses['train']:.4f}, val loss: {losses['valid']:.4f}")
            
            x, y = x.to(device), y.to(device)
            logits, loss = model(x, y)
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()


In [8]:
# Setting up the dataset parameters
### (Option 1) We can use different tokenizer, like SentencePiece
### (Option 2) We can build our own tokenizer, using huggingface library
import tiktoken



file_path = "poe_data.txt"

# Reading file, preparing tokenizer
with open(file_path, 'r', encoding="utf-8") as f:
            text = f.read()

# Setting up dataset
split_ratio = 0.9
context_length = 256
offset = 1  # I am wondering what would be the results for 2, for example
custom_tokenizer = False

if custom_tokenizer:
    tokenizer = Tokenizer(text)
    vocab_size = tokenizer.vocab_size
else:
    tokenizer = tiktoken.get_encoding("cl100k_base")
    vocab_size = tokenizer.n_vocab

# Setting up model (!!! head_size should be n_embed // head_num)
net_config = { "vocab_size": vocab_size,
               "n_embed": 32,
               "context_length": context_length,
               "att_head_num": 4,
               "blocks_num": 3,
               "dropout": .2}

symbols_limit = 50
model = OnlyDecoder(net_config)
model.to(device)

# Training parameters
hypers = {
    "lr": .3e-4,
    "epochs": 2,
    "batch_size": 8,
    "eval_per_epoch": 10,
    "eval_iterations": 200,
    "break_iter": None
}

# Training
train_set = PoeDataset(text, 'train', split_ratio, context_length, tokenizer, offset=offset)
val_set = PoeDataset(text, 'valid', split_ratio, context_length, tokenizer, offset=offset)

train_model(model, train_set, val_set, hypers, device=device)

Epoch: [1/2] Step: [0/48603], train loss: 11.6390, val loss: 11.6448
Epoch: [1/2] Step: [4860/48603], train loss: 7.0154, val loss: 7.0475
Epoch: [1/2] Step: [9720/48603], train loss: 6.8036, val loss: 6.8972
Epoch: [1/2] Step: [14580/48603], train loss: 6.6562, val loss: 6.7863
Epoch: [1/2] Step: [19440/48603], train loss: 6.5063, val loss: 6.7199
Epoch: [1/2] Step: [24300/48603], train loss: 6.3745, val loss: 6.6629
Epoch: [1/2] Step: [29160/48603], train loss: 6.2774, val loss: 6.5777
Epoch: [1/2] Step: [34020/48603], train loss: 6.2041, val loss: 6.5671
Epoch: [1/2] Step: [38880/48603], train loss: 6.1158, val loss: 6.5030
Epoch: [1/2] Step: [43740/48603], train loss: 6.0360, val loss: 6.4572
Epoch: [1/2] Step: [48600/48603], train loss: 5.9720, val loss: 6.4426
Epoch: [2/2] Step: [0/48603], train loss: 5.9778, val loss: 6.4123
Epoch: [2/2] Step: [4860/48603], train loss: 5.9019, val loss: 6.3775
Epoch: [2/2] Step: [9720/48603], train loss: 5.8399, val loss: 6.3521
Epoch: [2/2] Ste

In [9]:
save_path = 'state.pt'
torch.save(model.state_dict(), save_path)

In [10]:
test_text_length = 200

# Test it
starter = torch.zeros((1,1), dtype=torch.long, device=device)
print(tokenizer.decode([t.item() for t in model.generate_new_text(starter, test_text_length)]))

—hung breadth to my beloved:—ithum!—hisuation can be considered with his explanations.   “Oiluded been too say dat left. In each time known for human diminutive facacy of the besides in the immediate comensance, I had been attributed ago (for indère. “shot—thereical neglected did he called into it? I have a thing When and “The good trance-ocityings.   Meumies of guggestion.”   “Independ have reached me!”! do the escape!” overs roundless fellow, and tried in some a young from the same portion to me the correctok. “unlessaries,” That, King, and I youasora, madis; all ‘Be nor emerged—that it dust, boldly. Oaphise in a “perhaps, will sustain the sleeve than his drawing, “van proverbiter rather vast coatieur’é and origin, moreover indul insin, as the balloon were now die. This end, will
